In [2]:
import requests, time
import pandas as pd
import time
import re
import sqlalchemy
import os
from datetime import date
import numpy as np
import statistics

In [3]:
places = pd.read_csv('data/coord.csv')
places.rename(columns = {'Unnamed: 0':'location'},inplace=True)

In [4]:
places['coord'] = places['latitude'].round(3).astype(str) +','+ places["longitude"].round(3).astype(str)

In [5]:
places.index = places['location']
places.drop(columns = ['coastDistance','latitude','longitude','location'],inplace=True)


In [6]:
places.coord

location
Albury               -36.08,146.916
BadgerysCreek       -33.882,150.744
Cobar               -31.498,145.834
CoffsHarbour        -30.296,153.114
Moree               -29.462,149.841
Newcastle            -32.919,151.78
NorahHead           -33.282,151.568
NorfolkIsland       -29.033,167.948
Penrith             -33.751,150.694
Richmond             -37.82,145.003
Sydney               -33.87,151.208
SydneyAirport       -33.935,151.166
WaggaWagga          -35.115,147.368
Williamtown         -32.815,151.843
Wollongong          -34.428,150.893
Canberra            -35.298,149.101
Tuggeranong         -35.421,149.092
MountGinini          -35.53,148.773
Ballarat            -37.562,143.861
Bendigo             -36.759,144.283
Sale                -38.105,147.065
MelbourneAirport    -37.667,144.833
Melbourne           -37.814,144.963
Mildura             -34.185,142.162
Nhil                -35.433,141.283
Portland            -38.346,141.604
Watsonia            -37.711,145.084
Dartmoor           

In [7]:
coord = places.to_dict()['coord']

In [8]:
coord

{'Albury': '-36.08,146.916',
 'BadgerysCreek': '-33.882,150.744',
 'Cobar': '-31.498,145.834',
 'CoffsHarbour': '-30.296,153.114',
 'Moree': '-29.462,149.841',
 'Newcastle': '-32.919,151.78',
 'NorahHead': '-33.282,151.568',
 'NorfolkIsland': '-29.033,167.948',
 'Penrith': '-33.751,150.694',
 'Richmond': '-37.82,145.003',
 'Sydney': '-33.87,151.208',
 'SydneyAirport': '-33.935,151.166',
 'WaggaWagga': '-35.115,147.368',
 'Williamtown': '-32.815,151.843',
 'Wollongong': '-34.428,150.893',
 'Canberra': '-35.298,149.101',
 'Tuggeranong': '-35.421,149.092',
 'MountGinini': '-35.53,148.773',
 'Ballarat': '-37.562,143.861',
 'Bendigo': '-36.759,144.283',
 'Sale': '-38.105,147.065',
 'MelbourneAirport': '-37.667,144.833',
 'Melbourne': '-37.814,144.963',
 'Mildura': '-34.185,142.162',
 'Nhil': '-35.433,141.283',
 'Portland': '-38.346,141.604',
 'Watsonia': '-37.711,145.084',
 'Dartmoor': '-27.996,115.189',
 'Brisbane': '-27.469,153.023',
 'Cairns': '-16.921,145.772',
 'GoldCoast': '-28.002,15

In [9]:
coord['Albury']

'-36.08,146.916'

In [32]:
api_key = open('apikey.txt', 'r').read()
day = '2022-12-15'
q = '-29.033,167.948'
url = f'http://api.worldweatheronline.com/premium/v1/past-weather.ashx?query={q}&date={day}&format=json&key=d8d884e4d04241d3832173923221511'
#url = f'http://api.weatherapi.com/v1/history.json?key={api_key}&q={q}&dt={day}'.replace("'","")

In [11]:
def rain_fall():
    chuva = []
    for i in range(8):
        chuva.append(float(response.json()['data']['weather'][0]['hourly'][i]['precipMM']))
    chuva = np.array(chuva)
    return np.mean(chuva)

In [13]:
def gust_s():
    gust_s = []
    for i in range(8):
        gust_s.append(float(response.json()['data']['weather'][0]['hourly'][i]['WindGustKmph']))
    gust_s = np.array(gust_s)
    return np.mean(gust_s)

In [14]:
def get_humidity():
    hum = []
    for i in range(8):
        hum.append(float(response.json()['data']['weather'][0]['hourly'][i]['humidity']))
    hum = np.array(hum)
    return np.mean(hum)

In [15]:
def cloud(hour):
    pc = float(response.json()['data']['weather'][0]['hourly'][hour]['cloudcover'])
    return np.round((pc*8)/100,0)
    

In [16]:
def rain_td(rainfall):
    if rainfall != 0.0:
        return 'Yes'
    else:
        return 'No'

In [17]:
def gust_d():
    gust_d = []
    for i in range(8):
        gust_d.append(response.json()['data']['weather'][0]['hourly'][i]['winddir16Point'])
    return statistics.mode(gust_d)

In [18]:
response = requests.get(url)
response

<Response [200]>

In [19]:
rain = pd.DataFrame(columns = ['date', 'location', 'mintemp', 'maxtemp', 'rainfall', 'evaporation',
       'sunshine', 'humidity9am', 'humidity3pm', 'pressure9am', 'pressure3pm',
       'cloud9am', 'cloud3pm', 'temp9am', 'temp3pm', 'raintoday',
       'raintomorrow', 'temp', 'humidity', 'precipitation3pm',
       'precipitation9am', 'wind_gustdir', 'wind_gustspeed', 'wind_dir9am',
       'wind_dir3pm', 'wind_speed9am', 'wind_speed3pm'])

In [20]:
rain['location'] = places.index
rain['date'] = day

In [21]:
features{'sunshine'}

SyntaxError: invalid syntax (4244737010.py, line 1)

In [22]:
sunshine = response.json()['data']['weather'][0]['sunHour']
mintemp = response.json()['data']['weather'][0]['mintempC']
maxtemp = response.json()['data']['weather'][0]['maxtempC']
rainfall = rainfall()
humidity9am = response.json()['data']['weather'][0]['hourly'][3]['humidity']
humidity3pm = response.json()['data']['weather'][0]['hourly'][5]['humidity']
pressure9am = response.json()['data']['weather'][0]['hourly'][3]['pressure']
pressure3pm = response.json()['data']['weather'][0]['hourly'][5]['pressure']
cloud9am = cloud(3)
cloud3pm = cloud(5)
temp9am = response.json()['data']['weather'][0]['hourly'][3]['tempC']
temp3pm = response.json()['data']['weather'][0]['hourly'][5]['tempC']
raintoday = raind_td(rainfall)
temp = response.json()['data']['weather'][0]['avgtempC']
humidity = humidity()
precipitation3pm = response.json()['data']['weather'][0]['hourly'][3]['precipMM']
precipitation9am = response.json()['data']['weather'][0]['hourly'][5]['precipMM']
wind_gustdir = gust_d()
wind_gustspeed = gust_s()
wind_dir9am = response.json()['data']['weather'][0]['hourly'][3]['winddir16Point']
wind_dir3pm = response.json()['data']['weather'][0]['hourly'][5]['winddir16Point']
wind_speed9am = response.json()['data']['weather'][0]['hourly'][3]['windspeedKmph']
wind_speed3pm = response.json()['data']['weather'][0]['hourly'][5]['windspeedKmph']

KeyError: 'weather'

In [36]:
features = ['sunshine','mintemp','maxtemp','rainfall','humidity9am','humidity3pm','pressure9am','pressure3pm','cloud9am','cloud3pm'
           'temp9am','temp3pm','raintoday','temp','humidity','precipitation3pm','precipitation9am','wind_gustdir','wind_gustspeed',
           'wind_dir9am','wind_dir3pm','wind_speed9am','wind_speed3pm']

In [ ]:
sunshine = response.json()['data']['weather'][0]['sunHour']
    mintemp = response.json()['data']['weather'][0]['mintempC']
    maxtemp = response.json()['data']['weather'][0]['maxtempC']
    rainfall = rainfall()
    humidity9am = response.json()['data']['weather'][0]['hourly'][3]['humidity']
    humidity3pm = response.json()['data']['weather'][0]['hourly'][5]['humidity']
    pressure9am = response.json()['data']['weather'][0]['hourly'][3]['pressure']
    pressure3pm = response.json()['data']['weather'][0]['hourly'][5]['pressure']
    cloud9am = cloud(3)
    cloud3pm = cloud(5)
    temp9am = response.json()['data']['weather'][0]['hourly'][3]['tempC']
    temp3pm = response.json()['data']['weather'][0]['hourly'][5]['tempC']
    raintoday = rain_td(rainfall)
    temp = response.json()['data']['weather'][0]['avgtempC']
    humidity = humidity()
    precipitation3pm = response.json()['data']['weather'][0]['hourly'][5]['precipMM']
    precipitation9am = response.json()['data']['weather'][0]['hourly'][3]['precipMM']
    wind_gustdir = gust_d()
    wind_gustspeed = gust_s()
    wind_dir9am = response.json()['data']['weather'][0]['hourly'][3]['winddir16Point']
    wind_dir3pm = response.json()['data']['weather'][0]['hourly'][5]['winddir16Point']
    wind_speed9am = response.json()['data']['weather'][0]['hourly'][3]['windspeedKmph']
    wind_speed3pm = response.json()['data']['weather'][0]['hourly'][5]['windspeedKmph']

In [94]:
for key in coord.keys():
    q=coord[key]
    url = f'http://api.worldweatheronline.com/premium/v1/past-weather.ashx?query={q}&date={day}&format=json&key={api_key}'
    
    response = requests.get(url)
    print(response)
    time.sleep(2)
    
    try:      
            features = {'sunshine': response.json()['data']['weather'][0]['sunHour'],
                        'mintemp' : response.json()['data']['weather'][0]['mintempC'],
                        'maxtemp' : response.json()['data']['weather'][0]['maxtempC'],
                       'rainfall' : rain_fall(),
                       'humidity9am': response.json()['data']['weather'][0]['hourly'][3]['humidity'],
                       'humidity3pm': response.json()['data']['weather'][0]['hourly'][5]['humidity'],
                       'pressure9am': response.json()['data']['weather'][0]['hourly'][3]['pressure'],
                       'pressure3pm': response.json()['data']['weather'][0]['hourly'][5]['pressure'],
                       'cloud9am': cloud(3),
                       'cloud3pm': cloud(5),
                       'temp9am': response.json()['data']['weather'][0]['hourly'][3]['tempC'],
                       'temp3pm': response.json()['data']['weather'][0]['hourly'][5]['tempC'],
                       'raintoday':rain_td(rainfall),
                       'temp': response.json()['data']['weather'][0]['avgtempC'],
                       'humidity': get_humidity(),
                       'precipitation3pm':response.json()['data']['weather'][0]['hourly'][5]['precipMM'],
                       'precipitation9am':response.json()['data']['weather'][0]['hourly'][3]['precipMM'],
                       'wind_gustdir' : gust_d(),
                        'wind_gustspeed': gust_s(),
                        'wind_dir9am': response.json()['data']['weather'][0]['hourly'][3]['winddir16Point'],
                        'wind_dir3pm': response.json()['data']['weather'][0]['hourly'][5]['winddir16Point'],
                        'wind_speed9am': response.json()['data']['weather'][0]['hourly'][3]['windspeedKmph'],
                        'wind_speed3pm': response.json()['data']['weather'][0]['hourly'][5]['windspeedKmph']
               }
    
            for i in features.keys():
                rain.loc[key,i]= features[i]
        
    except KeyError:
        continue
        
        

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [95]:
rain

,date,location,mintemp,maxtemp,rainfall,evaporation,sunshine,humidity9am,humidity3pm,pressure9am,...,humidity,precipitation3pm,precipitation9am,wind_gustdir,wind_gustspeed,wind_dir9am,wind_dir3pm,wind_speed9am,wind_speed3pm,cloud3pmtemp9am
location,,,,,,,,,,,,,,,,,,,,,
Albury,2022-12-14,Albury,6,15,0.0375,NaN,11.3,55,53,1013,...,68.125,0.0,0.0,SW,20.875,WSW,SW,17,21,cloud3pmtemp9am
BadgerysCreek,2022-12-14,BadgerysCreek,13,25,0.0,NaN,14.5,34,16,1008,...,37.625,0.0,0.0,WSW,18.125,WSW,SW,19,23,NaN
Cobar,2022-12-14,Cobar,14,22,0.0,NaN,11.6,36,18,1015,...,32.125,0.0,0.0,S,32.625,S,SSW,28,28,NaN
CoffsHarbour,2022-12-14,CoffsHarbour,17,25,0.0,NaN,14.1,43,48,1007,...,63.375,0.0,0.0,SSE,17.5,SSE,ESE,6,15,NaN
Moree,2022-12-14,Moree,18,28,0.0,NaN,14.0,25,20,1010,...,24.0,0.0,0.0,SSW,30.0,SW,SW,21,35,NaN
Newcastle,2022-12-14,Newcastle,15,26,0.0,NaN,14.2,35,28,1007,...,45.875,0.0,0.0,W,21.75,W,SSW,9,22,NaN
NorahHead,2022-12-14,NorahHead,14,25,0.0,NaN,14.2,33,21,1007,...,42.75,0.0,0.0,W,18.0,WSW,SW,13,18,NaN
NorfolkIsland,2022-12-14,NorfolkIsland,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Penrith,2022-12-14,Penrith,13,25,0.0,NaN,14.5,34,16,1008,...,37.625,0.0,0.0,WSW,18.125,WSW,SW,19,23,NaN


In [23]:
rain.index = rain['location']

In [26]:
rain.loc['Albury','mintemp'] = 0

In [31]:
rain.columns

Index(['date', 'location', 'mintemp', 'maxtemp', 'rainfall', 'evaporation',
       'sunshine', 'humidity9am', 'humidity3pm', 'pressure9am', 'pressure3pm',
       'cloud9am', 'cloud3pm', 'temp9am', 'temp3pm', 'raintoday',
       'raintomorrow', 'temp', 'humidity', 'precipitation3pm',
       'precipitation9am', 'wind_gustdir', 'wind_gustspeed', 'wind_dir9am',
       'wind_dir3pm', 'wind_speed9am', 'wind_speed3pm'],
      dtype='object')

In [90]:
response.json()

{'data': {'error': [{'msg': 'Unable to find any matching weather location to the query submitted!'}]}}

In [103]:
rain['raintoday']=rain.apply(lambda x: rain_td(x['rainfall']),axis=1)

In [23]:
df_test= pd.DataFrame(columns = ['date','location','rainfall','raintoday'])

In [24]:
df_test['date'] = '2022-12-15'
df_test['location'] = places.index
df_test.index = df_test['location']

In [33]:
url = f"http://api.worldweatheronline.com/premium/v1/past-weather.ashx?query={coord['Melbourne']}&date={day}&format=json&key={api_key}"

In [35]:
response = requests.get(url)

In [36]:
response.json()

{'data': {'request': [{'type': 'LatLon',
    'query': 'Lat -37.81 and Lon 144.96'}],
  'weather': [{'date': '2022-12-15',
    'astronomy': [{'sunrise': '05:52 AM',
      'sunset': '08:38 PM',
      'moonrise': '01:02 AM',
      'moonset': '11:50 AM',
      'moon_phase': 'Last Quarter',
      'moon_illumination': '48'}],
    'maxtempC': '16',
    'maxtempF': '61',
    'mintempC': '10',
    'mintempF': '51',
    'avgtempC': '14',
    'avgtempF': '58',
    'totalSnow_cm': '0.0',
    'sunHour': '11.4',
    'uvIndex': '3',
    'hourly': [{'time': '0',
      'tempC': '11',
      'tempF': '51',
      'windspeedMiles': '11',
      'windspeedKmph': '18',
      'winddirDegree': '196',
      'winddir16Point': 'SSW',
      'weatherCode': '176',
      'weatherIconUrl': [{'value': 'https://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0025_light_rain_showers_night.png'}],
      'weatherDesc': [{'value': 'Patchy rain possible'}],
      'precipMM': '0.1',
      'precipInches': '0.0',
   

In [37]:
rain_fall()

0.037500000000000006

In [40]:
for key in coord.keys():
    day = '2022-12-15'
    q=coord[key]
    url = f'http://api.worldweatheronline.com/premium/v1/past-weather.ashx?query={q}&date={day}&format=json&key={api_key}'
    
    response = requests.get(url)
    print(response)
    time.sleep(2)
    
    try:
            test={'rainfall' : rain_fall()}
            
            for i in test.keys():
                df_test.loc[key,i]= test[i]
    
    except KeyError:
        continue
    

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [41]:
df_test

,date,location,rainfall,raintoday
location,,,,
Albury,NaN,Albury,0.0,NaN
BadgerysCreek,NaN,BadgerysCreek,0.0,NaN
Cobar,NaN,Cobar,0.0,NaN
CoffsHarbour,NaN,CoffsHarbour,0.05,NaN
Moree,NaN,Moree,0.0,NaN
Newcastle,NaN,Newcastle,0.0125,NaN
NorahHead,NaN,NorahHead,0.0,NaN
NorfolkIsland,NaN,NorfolkIsland,NaN,NaN
Penrith,NaN,Penrith,0.0,NaN


In [44]:
df_test['raintoday']=df_test.apply(lambda x: rain_td(x['rainfall']),axis=1)

In [46]:
df_test['date'] = day

In [47]:
df_test

,date,location,rainfall,raintoday
location,,,,
Albury,2022-12-15,Albury,0.0,No
BadgerysCreek,2022-12-15,BadgerysCreek,0.0,No
Cobar,2022-12-15,Cobar,0.0,No
CoffsHarbour,2022-12-15,CoffsHarbour,0.05,Yes
Moree,2022-12-15,Moree,0.0,No
Newcastle,2022-12-15,Newcastle,0.0125,Yes
NorahHead,2022-12-15,NorahHead,0.0,No
NorfolkIsland,2022-12-15,NorfolkIsland,NaN,Yes
Penrith,2022-12-15,Penrith,0.0,No


In [48]:
df_test.to_csv('data/rain_15_12')